# 

In [3]:
import bluesky
import bluesky.plan_stubs as bps
from apstools.synApps import SscanRecord, SaveData
from ophyd import Device, EpicsSignal, EpicsSignalRO, Component, EpicsMotor
from apstools.plans import run_blocking_function
from ophyd.status import Status
import numpy as np
from epics import caput, caget
import logging
import time
import h5py
import os
import sys
import subprocess
import pathlib
# os.environ.pop('QT_QPA_PLATFORM', None)
# os.environ.pop('QT_QPA_PLATFORM_PLUGIN_PATH', None)
# sys.path.append("/home/beams/STAFF19ID/bluesky/")
# sys.path.append(str(pathlib.Path(__file__).absolute().parent))
from isn_instrument.devices.positionerstream_device import PositionerStream
from isn_instrument.devices.profile_move_device import ProfileMove
from isn_instrument.devices.softgluezynq_device import SoftGlueZynq
from isn_instrument.devices.tetramm_device import TetraMM
from isn_instrument.devices.xspress3_device import Xspress3
from isn_instrument.callbacks.trajectories import *

In [ ]:
def mkdir(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
            print(f"Created directory: {directory}")
        except OSError as e:
            print(f"Failed to create directory: {directory} - {e}")
    else:
        print(f"Directory already exists: {directory}")

In [ ]:
def mksubdirs(save_path, subdirs=[]):
    for folder in subdirs:
        path = f"{save_path}{folder}"
        mkdir(path)

In [ ]:
def pvget(pv):
    # TODO: refactor to use pvapy package - https://epics.anl.gov/extensions/pvaPy/production/index.html
    try:
        cmd = f'pvget {pv}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e


In [ ]:
def pvput(pv, value):
    # TODO: refactor to use pvapy package - https://epics.anl.gov/extensions/pvaPy/production/index.html
    try:
        cmd = f'pvput {pv} {value}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e

In [ ]:
def create_master_file(basedir, sample_name, groups = ["xspress3", "eiger", "mda", "tetramm", "positions"]):
    with h5py.File(f"{basedir}/{sample_name}_master.h5", "w") as f:
        for group in groups:
            f.create_group(group)
            files = os.listdir(f"{basedir}/{group}")
            if group == "tetramm":
                files = [file for file in files if file.split(".")[-1]=="nc"]
                string_data = [s.encode('utf-8') for s in files]
                dset = f[group].create_dataset("fnames", data=string_data)
                # print(string_data)
            elif group == "mda":
                files = [file for file in files if file.split(".")[-1]=="mda"]
                string_data = [s.encode('utf-8') for s in files]
                dset = f[group].create_dataset("fnames", data=string_data)
                # print(string_data)
            elif group == "xspress3": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                for file in files:
                    print(f"./{group}/{file}")
                    f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/entry")
            elif group == "positions": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                for file in files:
                    print(f"./{group}/{file}")
                    f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/stream")

In [ ]:
# Assumes file is in same folder as this notebook
%run -i ./run_scan_plan.py

## !!!! Instability notes

- if Profile move gets too many points (more than MAX_ARR_Something.. ) the ioc crashes,
  and the tetraMM will need to be power cycled

- Consecutive capture/stop acquire/stop commands seem to cause xspress3 ioc to crash.. need some recovery method. I think it
  is from triggering "erase" twice in a row; set erase on start to no.

- xspress3 buffer offloading still painfully slow.. for demo keep aq rate to less than 100Hz

- stop capture sometimes stops working on both xspress3 and tetramm: this was due to micdata file storage instability.

In [ ]:
RE = bluesky.RunEngine()
RE(run_scan(scan_type="fly", trajectory="snake", loop1="2idsft:m1", loop2="2idsft:m2", dwell_time="10", 
             devices={"flyXRF":"XSP3_1Chan:", "tetramm:":"2idsft:TetrAMM1", "profilemove":"2idsft:pm1", "softglue": "2idMZ1:", "positions":"posvr:"},
             sample_name="sample_name", pi_directory = "/mnt/micdata1/save_dev/", comments="",
             l1_center=0, l1_size=0.01, l1_width=0.5, l2_center=0, l2_size=0.01, l2_width=0.5,
             ):